In [ ]:
import sys
sys.path.append("..") # Ensure the parent directory is in the path

import core.models.AssignmentBaseModel as BaseModel
import core
from core.DataLoader import DataPreprocessor
import core.AssignmentKFold as KFold
import numpy as np
from importlib import reload
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
MAX_JETS = 4
PLOTS_DIR = f"plots/"
import os
if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)

DataProcessor = DataPreprocessor(jet_features=["ordered_jet_pt", "ordered_jet_e", "ordered_jet_phi", "ordered_jet_eta", "ordered_jet_b_tag","m_l1j", "m_l2j", "dR_l1j", "dR_l2j"], 
                                lepton_features=["lep_pt","lep_e", "lep_eta", "lep_phi"],
                                jet_truth_label="ordered_event_jet_truth_idx", 
                                lepton_truth_label="event_lepton_truth_idx", 
                                global_features = ["met_met_NOSYS","met_phi_NOSYS"], 
                                max_leptons=2, 
                                max_jets = MAX_JETS, 
                                non_training_features= ["truth_ttbar_mass", "truth_ttbar_pt", "N_jets"], 
                                event_weight="weight_mc_NOSYS")
DataProcessor.load_data("/data/dust/group/atlas/ttreco/full_training.root", "reco", max_events=100000)
DataProcessor.prepare_data()
DataProcessor.normalise_data()
DataProcessor.split_data(0.2)

FileNotFoundError: [Errno 2] No such file or directory: '/data/dust/group/atlas/ttreco/full_training.root'

In [1]:
import core.components as components
class TransformerJetMatcher(BaseModel.AssignmentBaseModel):
    """
    Transformer-based model for jet-lepton assignment.

    Inherits from:
        BaseModel.AssignmentBaseModel: Base class for assignment models.
    """
    def __init__(self, data_processor):
        """
        Initializes the TransformerJetMatcher with a data processor.

        Args:
            data_processor (DataPreprocessor): An instance of DataPreprocessor for handling data loading
                                               and preprocessing.
        """
        super().__init__(data_processor)
        self.model = None

    def build_model(self, hidden_size = 64, n_heads = 8, stack_size = 3, ff_dim = None, dropout = 0.1):
        """
        Builds the Transformer-based model architecture.

        Args:
            hidden_size (int): Dimensionality of the embedding space.
            n_heads (int): Number of attention heads in the multi-head attention layer.
            stack_size (int): Number of Transformer blocks to stack.
            ff_dim (int): Dimensionality of the feed-forward network. If None, defaults to 4 * hidden_size.
            dropout (float): Dropout rate for regularization.
        """

        jet_input = keras.Input(shape=(self.max_jets, len(self.data_processor.jet_features)), name="jet_input")
        lepton_input = keras.Input(shape=(self.max_leptons, len(self.data_processor.lepton_features)), name="lepton_input")
        global_input = keras.Input(shape=(len(self.n_global),), name="global_input")

        # Masking layers to handle variable-length inputs
        jet_mask = components.GenerateMask(padding_value=self.padding_value)(jet_input)

        lepton_repeated = keras.layers.RepeatVector(self.max_jets)(lepton_input)
        global_repeated = keras.layers.RepeatVector(self.max_jets)(global_input)

        x = keras.layers.Concatenate(axis=-1)([jet_input, lepton_repeated, global_repeated])

        input_embedding = components.MLP([hidden_size],num_layers=3, activation="relu", dropout_rate=dropout)(x)
        x = keras.layers.Dropout(dropout)(input_embedding)
        for _ in range(stack_size):
            x = components.SelfAttentionBlock(hidden_size, n_heads, ff_dim or 4*hidden_size, dropout)(x, jet_mask)
        x = keras.layers.Dropout(dropout)(x)
        x = components.MLP(2, num_layers=4, dropout_rate=dropout)(x)
        output = keras.layers.Softmax(axis=1, name="assignment_output")(x, mask=jet_mask)
        self.model = keras.Model(inputs=[jet_input, lepton_input, global_input], outputs=output)



ModuleNotFoundError: No module named 'core'

In [ ]:

reload(KFold)

TransformerMatcher = TransformerJetMatcher(DataProcessor); TransformerMatcher.load_data(*DataProcessor.get_data())
TransformerMatcher.build_model(

)
TransformerMatcher.compile_model(
    lambda_excl=0, optimizer=keras.optimizers.Adam(learning_rate=0.001)
)
TransformerMatcher.model.summary()
TransformerMatcher.load_data(*DataProcessor.get_data())

In [ ]:
TransformerMatcher.train_model(epochs=100,
                                batch_size=128,
                                verbose=1,
                                weight = "sample",
                                callbacks = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=50, restore_best_weights=True, mode ="max"))


In [ ]:
TransformerMatcher.plot_history()

In [ ]:
TransformerMatcher.plot_confusion_matrix()

In [ ]:
TransformerMatcher.plot_permutation_importance(shuffle_number=5)